In [1]:
%load_ext autoreload
%autoreload 2
"""
    Some imports used throughout the notebook
"""
import time

from visualize import *

from cpmpy.transformations.normalize import toplevel_list
from factory import *
from read_data import get_data

import numpy as np
np.set_printoptions(linewidth=90)
# preload solvers
from cpmpy import SolverLookup
names = SolverLookup.solvernames()

def get_avg_outputlits(seq):
    lits = sum(len(out.literals()) for _,_,out in seq[:-1])
    return lits / (len(seq)-1)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-09


## Deductive explanations in constraint solving - MUS and beyond
### _Ignace Bleukx_, Jo Devriendt, Bart Bogaerts, Dimos Tsouros, Tias Guns

<p>&nbsp;</p>

<table><tr style="background: white;">
    <td>&nbsp;</td>
    <td style="text-align: center; vertical-align: middle;"><img src="img/kul.jpg" width=40%></td>
    <td style="text-align: center; vertical-align: middle;"><img src="img/erc.jpg" width=45%></td>
</tr></table>

<!-- Thanks to Bart Bogaerts, Emilio Gamba and Jo Devriendt -->


<small>This presentation is an executable Jupyter notebook</small>

Link to slides and more examples: https://github.com/CPMpy/XCP-explain

## AI and Constraint Solving

<center><img src="img/intro_ai.png" width="65%" align="center" style="margin-top:10px"></center>


## Constraint Solving

<img src="img/solutions_vizual.png" width="45%" align="right" style="margin-top:100px">

Solving combinatorial optimization problems in AI

- Vehicle Routing

- Scheduling

- Manufacturing

- Other combinatorial problems ...

## Model + Solve

<center><img src="img/model_solve.png" width=70%></center>

- Reason over the constraints to find a solution
- Find the optimal solution according to the objective 
    - (maximize gain or minimize penalty)

## Explainable Constraint Programming (XCP)

In general, "**Why X?**" &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (with X a solution or UNSAT)

To be defined... 

## Explainable Constraint Programming (XCP)

In general, "**Why X?**" &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (with X a solution or UNSAT)

3 patterns:
- **Deductive explanation:**
  - _Why does X follow from the constraints?_
- Contrastive/Counterfactual explanation:
  - _Why X and not Z?_
- Conversational explanation:
  - _Iteratively refine explanation & model_


For more examples, visit https://github.com/CPMpy/XCP-explain

## Causal explanation, mode of interaction:

<center><img src="img/interaction_figure4.png" width=20%></center>

### Then what?

### Then what?

<img src="img/interaction_figure4.png" width="20%" align="left" style="margin:5%;"/>

1. User is happy with the answer <br>
    (e.g. better understands the problem)
2. User changes the answer and uses that <br>
    (solution interaction; no solver involvement)
3. User changes the model and reiterates <br>
    (e.g. better understands how to model the problem)
4. User interacts with interactive system <br>
    (e.g. conversational explanations)



## The model: Nurse Scheduling

<img src="img/nurse_rost_prob.jpg">

* The assignment of _shifts_ and _holidays_ to nurses.
* Each nurse has their own restrictions and preferences,
    as does the hospital.



## Nurse Rostering: data

Instances from http://www.schedulingbenchmarks.org/

"benchmark test instances from various sources including industrial collaborators and scientific publications."

<!-- 7 types of hospital constraints, 2 types of nurse constraints -->

In [2]:
#instance = "http://www.schedulingbenchmarks.org/nrp/data/Instance1.txt"
instance = "Benchmarks/Instance1.txt"
data = get_data(instance)

# all data is stored as DataFrame tables
data.staff

,# ID,MaxShifts,MaxTotalMinutes,MinTotalMinutes,MaxConsecutiveShifts,MinConsecutiveShifts,MinConsecutiveDaysOff,MaxWeekends,D,name
0,A,D=14,4320,3360,5,2,2,1,14,Megan
1,B,D=14,4320,3360,5,2,2,1,14,Katherine
2,C,D=14,4320,3360,5,2,2,1,14,Robert
3,D,D=14,4320,3360,5,2,2,1,14,Jonathan
4,E,D=14,4320,3360,5,2,2,1,14,William
5,F,D=14,4320,3360,5,2,2,1,14,Richard
6,G,D=14,4320,3360,5,2,2,1,14,Kristen
7,H,D=14,4320,3360,5,2,2,1,14,Kevin


In [3]:
print("Nr of days to schedule:", data.horizon)
print("Nr of shift types:", len(data.shifts))

pd.merge(data.days_off, data.staff[["# ID","name"]], left_on="EmployeeID", right_on="# ID", how="left")

Nr of days to schedule: 14
Nr of shift types: 1


,DayIdx,# ID,name
0,0,A,Megan
1,5,B,Katherine
2,8,C,Robert
3,2,D,Jonathan
4,9,E,William
5,5,F,Richard
6,1,G,Kristen
7,7,H,Kevin


## Nurse Rostering: constraints 1/2

### hospital constraints/preferences:

<img src="img/nurse_rost_prob.jpg" align="right">

* nb of nurses assigned
* max nb of shifts
* max nb of weekend shifts
* min nb of (consecutive) days off
* min/max minutes worked
* min/max consecutive shifts
* shift rotation


## Nurse Rostering: constraints 2/2

### nurse constraints/preferences:

<img src="img/nurse_rost_prob.jpg" align="right">

* specific days off-duty
* specific shift requests (on/off)


## CPMpy: http://cpmpy.readthedocs.io

We will use the CPMpy modeling library in Python for this presentation

<center><img src="img/cpmpy-intro.png" style="max-width: 70%;" /></center>
 

## The system: http://cpmpy.readthedocs.io

CPMpy is a Constraint Programming and Modeling library in Python, <br /> based on numpy, with direct solver access. <br /> 

**Features** used in this tutorial:

- Easy integration with visualisation tools (pandas, matplotlib)
- Object-oriented programming (constraints are Python objects we can create, copy, update)
- Repeatedly solving subsets of constraints (assumption variables)
- Incremental solving (e.g. SAT, MIP/Hitting set)



## Nurse Rostering in CPMpy

### Variables: 

assignment of shift types (_0=none_) to nurses

In [4]:
nurse_view = cp.intvar(0, len(data.shifts), # lb, ub
                       shape=(len(data.staff), data.horizon),
                       name="nv")
nurse_view

NDVarArray([[nv[0,0], nv[0,1], nv[0,2], nv[0,3], nv[0,4], nv[0,5], nv[0,6], nv[0,7],
             nv[0,8], nv[0,9], nv[0,10], nv[0,11], nv[0,12], nv[0,13]],
            [nv[1,0], nv[1,1], nv[1,2], nv[1,3], nv[1,4], nv[1,5], nv[1,6], nv[1,7],
             nv[1,8], nv[1,9], nv[1,10], nv[1,11], nv[1,12], nv[1,13]],
            [nv[2,0], nv[2,1], nv[2,2], nv[2,3], nv[2,4], nv[2,5], nv[2,6], nv[2,7],
             nv[2,8], nv[2,9], nv[2,10], nv[2,11], nv[2,12], nv[2,13]],
            [nv[3,0], nv[3,1], nv[3,2], nv[3,3], nv[3,4], nv[3,5], nv[3,6], nv[3,7],
             nv[3,8], nv[3,9], nv[3,10], nv[3,11], nv[3,12], nv[3,13]],
            [nv[4,0], nv[4,1], nv[4,2], nv[4,3], nv[4,4], nv[4,5], nv[4,6], nv[4,7],
             nv[4,8], nv[4,9], nv[4,10], nv[4,11], nv[4,12], nv[4,13]],
            [nv[5,0], nv[5,1], nv[5,2], nv[5,3], nv[5,4], nv[5,5], nv[5,6], nv[5,7],
             nv[5,8], nv[5,9], nv[5,10], nv[5,11], nv[5,12], nv[5,13]],
            [nv[6,0], nv[6,1], nv[6,2], nv[6,3], nv[6,4], 

### Constraints:

Specific days off-duty

In [5]:
for (empl_id, row) in data.days_off.iterrows():
    empl_idx = data.staff.index[data.staff["# ID"] == empl_id][0]
    day_idx = row["DayIdx"]
    
    con = (nurse_view[empl_idx, day_idx] == 0)
    
    con.set_description(f"{data.staff.iloc[empl_idx]['name']} should not work on day {day_idx}")
    print("-",con)

- Megan should not work on day 0
- Katherine should not work on day 5
- Robert should not work on day 8
- Jonathan should not work on day 2
- William should not work on day 9
- Richard should not work on day 5
- Kristen should not work on day 1
- Kevin should not work on day 7


## Object-oriented Nurse Rostering CPMpy model factory

In [7]:
factory = NurseSchedulingFactory(data)
model, nurse_view = factory.get_full_model()  # CPMpy model with all constraints
model.solve(solver="ortools")

True

In [8]:
nurse_view.value()

array([[0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]])

## Nurse rostering in CPMpy: visualisation, with pandas

In [ ]:
def visualize(sol, factory):
    weeks = [f"Week {i + 1}" for i in range(factory.data.horizon // 7)]
    weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    nurses = factory.data.staff['name'].tolist()

    df = pd.DataFrame(sol,
                      columns=pd.MultiIndex.from_product((weeks, weekdays)),
                      index=factory.data.staff.name)
    mapping = factory.idx_to_name
    df = df.map(lambda v: mapping[v] if v is not None and v < len(mapping) else '')  # convert to shift names

    for shift_type in factory.shift_name_to_idx:
        if shift_type == "F":continue
        sums = (df == shift_type).sum()  # cover for each shift type
        req = factory.data.cover["Requirement"][factory.data.cover["ShiftID"] == shift_type]
        req.index = sums.index
        df.loc[f'Cover {shift_type}'] = sums.astype(str) + "/" + req.astype(str)
    df["Total shifts"] = (df != "F").sum(axis=1)  # shifts done by nurse
    subset = (df.index.tolist()[:-len(factory.data.shifts)], df.columns[:-1])
    style = df.style.set_table_styles([{'selector': '.data', 'props': [('text-align', 'center')]},
                                       {'selector': '.col_heading', 'props': [('text-align', 'center')]},
                                       {'selector': '.col7', 'props': [('border-left',"2px solid black")]}])
    style = style.map(lambda v: 'border: 1px solid black', subset=subset)
    style = style.map(color_shift, factory=factory, subset=subset)  # color cells
    return style

from visualize import *

## Nurse rostering in CPMpy: visualisation, with pandas

In [9]:
visualize(nurse_view.value(), factory)

## Deductive Explanations in Constraint Solving

Explaing _why_ something follows from the constraints

We consider _why_ is there no solution?

## Converting to Decision model


<img src="img/explain_unsat.png" width="15%" align="left" style="margin:50px;">

* Model nurse rostering problem as decision problem <br>
    (no objective)
        
* Nurse **preferences** are also hard constraints

In [10]:
# model as decision model
factory = NurseSchedulingFactory(data)
model, nurse_view = factory.get_decision_model()  # CMPpy DECISION Model
model.solve()

False

... no solution found

In [11]:
constraints = toplevel_list(model.constraints, merge_and=False) # normalization for later
print(f"Model has {len(constraints)} constraints:")
for cons in constraints: print("-", cons)

Model has 168 constraints:
- Megan cannot work more than 14 shifts of type 1
- Katherine cannot work more than 14 shifts of type 1
- Robert cannot work more than 14 shifts of type 1
- Jonathan cannot work more than 14 shifts of type 1
- William cannot work more than 14 shifts of type 1
- Richard cannot work more than 14 shifts of type 1
- Kristen cannot work more than 14 shifts of type 1
- Kevin cannot work more than 14 shifts of type 1
- Megan cannot work more than 4320min
- Katherine cannot work more than 4320min
- Robert cannot work more than 4320min
- Jonathan cannot work more than 4320min
- William cannot work more than 4320min
- Richard cannot work more than 4320min
- Kristen cannot work more than 4320min
- Kevin cannot work more than 4320min
- Megan cannot work more than 3360min
- Katherine cannot work more than 3360min
- Robert cannot work more than 3360min
- Jonathan cannot work more than 3360min
- William cannot work more than 3360min
- Richard cannot work more than 3360min
-



<img src="img/mus.png" width="20%" align="left" style="margin:50px;">

Trim model to minimal set of constraints

... minimize cognitive burden for user


### How to compute a MUS?

Deletion-based MUS algorithm

_[Joao Marques-Silva. Minimal Unsatisfiability: Models, Algorithms and Applications. ISMVL 2010. pp. 9-14]_

In [12]:
def mus_naive(constraints):
    m = cp.Model(constraints)
    assert m.solve() is False, "Model should be UNSAT"
    
    core = constraints
    i = 0
    while i < len(core):
        subcore = core[:i] + core[i+1:]
        if cp.Model(subcore).solve() is True:
            i += 1 # removing makes it SAT, need to keep
        else:
            core = subcore # can safely delete 
    return core

### How to compute a MUS?

CPMpy implements an incremental version of this, using assumption variables

* `cpmpy.tools.mus`

In [14]:
from cpmpy.tools.mus import mus, mus_naive

solver = "ortools"
subset = mus(model.constraints, solver=solver) 

print("Length of MUS:", len(subset))
for cons in subset: print("-", cons)

Length of MUS: 11
- Shift D on Sat 1 must be covered by 5 nurses out of 8
- Robert can work at most 5 days before having a day off
- Kevin should work at most 1 weekends
- Katherine has a day off on Sat 1
- Richard has a day off on Sat 1
- Robert requests to work shift D on Mon 1
- Robert requests to work shift D on Tue 1
- Robert requests to work shift D on Wed 1
- Robert requests to work shift D on Thu 1
- Robert requests to work shift D on Fri 1
- Kevin requests to work shift D on Sun 2


In [15]:
visualize_constraints(subset, nurse_view, factory)

### Many MUS'es may exist...

_Liffiton, M.H., & Malik, A. (2013). Enumerating infeasibility: Finding multiple MUSes quickly. In
Proceedings of the 10th International Conference on Integration of AI and OR Techniques in Constraint
    Programming (CPAIOR 2013) (pp. 160–175)_

In [ ]:
# MARCO MUS/MSS enumeration
from explanations.marco_mcs_mus import do_marco
solver = "ortools"  # default solver
if "exact" in cp.SolverLookup.solvernames(): solver = "exact"  # fast for increment solving
    
t0 = time.time()
cnt = 0
for (kind, sset) in do_marco(model, solver=solver):
    if kind == "MUS":
        print("M", end="")
        cnt += 1
    else: print(".", end="") # MSS
    
    if time.time() - t0 > 20:  break  # for tutorial: break after 20s
print(f"\nFound {cnt} MUSes in", time.time() - t0)


### Many MUS'es may exist...

<img src="img/musses.png" width="40%" align="left" style="margin-left:50px; margin-right:50px">

This problem has just 168 constraints, yet 100.000+ MUSes exist...

Which one to show? 

In explanations less is more, so lets find the **smallest one directly!**

_Ignatiev, Alexey, et al. "Smallest MUS extraction with minimal hitting set dualization." International Conference on Principles and Practice of Constraint Programming. Cham: Springer International Publishing, 2015._

In [16]:
from explanations.subset import smus

small_subset = smus(model.constraints, solver="ortools", hs_solver="gurobi")

print("Length of sMUS:", len(small_subset))
for cons in small_subset:  
    print("-", cons)

Length of sMUS: 3
- Robert has a day off on Tue 2
- Richard requests to not work shift D on Tue 2
- Shift D on Tue 2 must be covered by 7 nurses out of 8


In [17]:
visualize_constraints(small_subset, nurse_view, factory)

## Step-wise explanation

<img src="img/explain_step-wise.png" width="20%" align="right" style="margin:50px;">

- We were lucky, the SMUS is pretty understandable
- What if its not?
- Disect the conflict into smaller steps <br>
  -> Step-wise Explanations

> Ignace Bleukx, Jo Devriendt, Emilio Gamba, Bart Bogaerts, Tias Guns. Simplifying Step-wise Explanation Sequences. 29th International Conference on Principles and Practice of Constraint Programming (CP23), 2023.

## Step-wise explanations

Find a sequence of small derivation steps, _explaining_ a goal set of literals (or a goal _fact_)


Current work: explain how to solve logic puzzles **with a single solution** 

> Bogaerts, Bart, Emilio Gamba, and Tias Guns. "A framework for step-wise explaining how to solve constraint satisfaction problems." Artificial Intelligence 300 (2021): 103550.

<img src="img/stepwise.png" width="100%" align="center" style="">


In that setting _no_ risk on redundancy as **every** literal (cell in the sudoku) needs to be explained

Imagine only a subset of the cells needs to be explained.

How to decide which cells to derive during the sequence? Maybe "uninteresting cells" to the user are usefull as intermediate result?

Also applies to the UNSAT case, on which we will focus for now

## 3-phase approach:

Greedy construction of the sequence
<img src="img/manysteps.png" width="80%" style="margin-bottom: -150px; margin-top: -50px"><br>
Removing redundant steps
<img src="img/littledirtysteps.png" width="80%" style="margin-bottom: -100px;  margin-top:-50px;">
Removing spurious input/output
 <img src="img/littlecleansteps.png" width="45%" style="margin-top:-120px">

###  1. Greedy construction

<img src="img/greedy_construct.png" width="50%" align="left" style="margin:20px;">



### Maximal propagation

Implemented in CPMpy and available incrementally in the Exact solver

In [18]:
from cpmpy.tools.maximal_propagate import maximal_propagate

x = cp.intvar(0,10, name="x")
y = cp.intvar(0,10, name="y")

maximal_propagate(x + y <= 3)

{x: {0, 1, 2, 3}, y: {0, 1, 2, 3}}

In [19]:
# or propagate multiple constraints at once
maximal_propagate([x + y <= 3, x + 2 == y])

{x: {0}, y: {2}}

## Greedy construction for the Nurse rostering problem

In [20]:
from explanations.stepwise import forward_construction
        
seq = forward_construction(model.constraints)
print(f"Found sequence of length {len(seq)}")

Found sequence of length 32


In [21]:
nurse_view.clear()
for step in seq:
    display(visualize_step(step, nurse_view, factory))

Propagating constraint: Richard requests to not work shift D on Tue 2


Propagating constraint: Robert requests to not work shift D on Sat 2


Propagating constraint: Katherine requests to work shift D on Thu 1


Propagating constraint: Katherine requests to work shift D on Fri 1


Propagating constraint: Kevin requests to not work shift D on Thu 1


...

Propagating constraint: Kevin can work at most 5 days before having a day off


Propagating constraint: Kevin has a day off on Mon 2


Propagating constraint: William should work at least 2 days before having a day off


Propagating constraint: Megan has a day off on Mon 1


Propagating constraint: Robert has a day off on Tue 2


\+ The good: <br>
- we get a sequence of explanation steps leading to the conflict!
- each step has low number of constraints

\- The bad: <br>
- the sequence is VERY long...

### 2. Filtering redundant steps

<img src="img/deletion_filter.png" width="50%" align="left" style="margin-right:40px;">

Main idea: leave out a step and check if the sequence is still valid

If yes: remove the step,

If no: we need the step in the sequence

In [22]:
from explanations.stepwise import backward_filtering

filtered = backward_filtering(model.constraints, seq)
print(f"Filtered sequence to length {len(filtered)}")
print("Avg nb of output literals:", get_avg_outputlits(filtered))

Filtered sequence to length 3
Avg nb of output literals: 4.5


In [23]:
nurse_view.clear()
for step in filtered:
    display(visualize_step(step, nurse_view, factory))

Propagating constraint: Richard requests to not work shift D on Tue 2


Propagating constraint: Shift D on Tue 2 must be covered by 7 nurses out of 8


Propagating constraint: Robert has a day off on Tue 2


\+ The good: <br>
- Our sequence is much shorter and easier to comprehend

\- The bad: <br>
- Individual steps may derive or use too much information

### 3. Simplify steps by removig input/output information

<img src="img/relaxation.png" width="50%" align="left" style="margin-right:40px;">

In [24]:
from explanations.stepwise import relax_sequence

relaxed = relax_sequence(filtered, time_limit=100)
print(f"Relaxed sequence has length {len(relaxed)}")
print("Avg nb of output literals:", get_avg_outputlits(relaxed))

Relaxed sequence has length 3
Avg nb of output literals: 1.0


In [25]:
nurse_view.clear()
for step in relaxed:
    display(visualize_step(step, nurse_view, factory))

Propagating constraint: Richard requests to not work shift D on Tue 2


Propagating constraint: Shift D on Tue 2 must be covered by 7 nurses out of 8


Propagating constraint: Robert has a day off on Tue 2


## Results on benchmarks

<img src="img/steps.png" width="80%">

<img src="img/inputlits.png" width="80%">

<img src="img/outputlits.png" width="80%">

# Conclusions

- Deductive explanations are a helpful means of explaining _why_ a problem is unsatisfiable

- Step-wise explanations can be applied to the problem of explaining unsatisfiability, but need to take care for redundancy



# Want to learn more?

<img src="img/qr-code.png" align="right" width="30%" style="margin-top:20px">

Tutorial with variety of explanation techniques recorded

Also includes _how_ to adapt the model to make it satisfiable 

(i.e., _contrastive_ explanations)

